In [56]:
SMALL_DATA_MODE = True
if SMALL_DATA_MODE:
    DATA_FILE = "data/smalldata/"
    datasize = 852
else:
    DATA_FILE = "data/"
    datasize = 852 #cambiar con el tamaño total de los datos

In [31]:
# PARAMETER
#-------------
# check if IS_MASTER exists, this variable will only exist if it's being called by MASTER notebook.
# if it does not exist, set it to False
try: IS_MASTER
except: IS_MASTER = False
# The code below will only run if it's NOT being called from MASTER notebook

if IS_MASTER == False:
    DATA_DIR = 'data/temp/' # 
    RAW_FILE = f'data/raw/smalldata/' # use "small data" here
    PROCESSED_FILE = f'{DATA_DIR}processed.pkl' # always use pickle for fast I/O!
    OTHER_PREPROCESS_PARAMETER = ... # e.g. batch size, sliding window size, etc

In [33]:
from xml.dom import minidom
import cv2 as cv2

In [34]:
def getText(nodelist):
    rc = []
    for node in nodelist:
        if node.nodeType == node.TEXT_NODE:
            rc.append(node.data)
    return ''.join(rc)

# Cropping the faces
We begin by cropping the faces from the training data. This would be useful to train the HOG detector.

In [14]:

counter=1
extension='.png'

for x in range(datasize):#refactorizar para poder iterar sobre un número indefinido de imágenes usando try + exceptions
    xmldoc = minidom.parse(DATA_FILE + 'annotations/maksssksksss'+str(x)+'.xml')
    img=cv2.imread(DATA_FILE + 'images/maksssksksss'+str(x)+'.png',0)
    objList = xmldoc.getElementsByTagName('object')
    for obj in objList:
        nameList=obj.getElementsByTagName('name')
        if(getText(nameList[0].childNodes)=='without_mask'):
            root= 'cropped/without_mask/img'
        elif (getText(nameList[0].childNodes)=='with_mask'):
            root= 'cropped/with_mask/img'
        else:
            root='cropped/wrong/img'
        xminList=obj.getElementsByTagName('xmin')
        yminList=obj.getElementsByTagName('ymin')
        xmaxList=obj.getElementsByTagName('xmax')
        ymaxList=obj.getElementsByTagName('ymax')
        crop_img = img[int(getText(yminList[0].childNodes)):int(getText(ymaxList[0].childNodes)), int(getText(xminList[0].childNodes)):int(getText(xmaxList[0].childNodes))]
        #crop_img = cv2.adaptiveThreshold(crop_img,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,11,2)
        cv2.imwrite(DATA_DIR + root +str(counter)+extension, crop_img)
        print('Processing: ' + str(counter) + 'of ' + str(datasize) + ';')
        counter=counter+1
print('Processing complete!')

In [62]:
import os
directory = DATA_FILE + 'temp/cropped/Si/'

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        make = filename.split("/")[-2]
        print(make)
        img=cv2.imread(directory+filename, 0)
        # load the image, convert it to grayscale, and detect edges
        #try:
            #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        edges = cv2.Canny(img,100,200)
        resized = cv2.resize(edges, (200, 100))
        path = DATA_FILE + 'temp/resized/'+ make + '/'
        extension = '.png'
        if not cv2.imwrite(path+ filename + extension, resized): 
            raise Exception("Could not write image")


In [66]:
import os
directory = DATA_FILE + 'temp/cropped/No/'

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        make = filename.split("/")[-1]
        print(make)
        img=cv2.imread(directory+filename, 0)
        # load the image, convert it to grayscale, and detect edges
        #try:
            #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        edges = cv2.Canny(img,100,200)
        resized = cv2.resize(edges, (200, 100))
        path = DATA_FILE + 'temp/resized/'+ make + '/'
        extension = '.png'
        if not cv2.imwrite(path+ filename + extension, resized): 
            raise Exception("Could not write image")


In [71]:
directory = DATA_FILE + 'temp/cropped/Mal/'

for filename in os.listdir(directory):
    if filename.endswith(".png"):
        make = directory.split("/")[-2]
        #print(make)
        img=cv2.imread(directory+filename, 0)
        # load the image, convert it to grayscale, and detect edges
        #try:
            #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        
        edges = cv2.Canny(img,100,200)
        resized = cv2.resize(edges, (200, 100))
        path = DATA_FILE + 'temp/resized/'+ make + '/'
        if not cv2.imwrite(path+ filename, resized): 
            raise Exception("Could not write image")

Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal
Mal


# HOG Descriptor

In [73]:
#definition of HOG descriptor

cell_size = (8, 8)  # h x w in pixels
block_size = (2, 2)  # h x w in cells
nbins = 9  # number of orientation bins

# winSize is the size of the image cropped to an multiple of the cell size
hog = cv2.HOGDescriptor(_winSize=(200 // cell_size[1] * cell_size[1],
                                  100 // cell_size[0] * cell_size[0]),
                        _blockSize=(block_size[1] * cell_size[1],
                                    block_size[0] * cell_size[0]),
                        _blockStride=(cell_size[1], cell_size[0]),
                        _cellSize=(cell_size[1], cell_size[0]),
                        _nbins=nbins)

n_cells = (100 // cell_size[0], 200 // cell_size[1])


In [77]:
import glob
import os
import numpy as np
samples = []
labels = []    

Si_path = "data/smalldata/temp/resized/Si/"
No_path = "data/smalldata/temp/resized/No/"
Mal_path = "data/smalldata/temp/resized/Mal/"
# Get positive samples
for filename in glob.glob(os.path.join(Si_path, '*.png')):
    img = cv2.imread(filename, 0)
    hist = hog.compute(img)
    samples.append(hist)
    labels.append(2)
    
# Get negative samples
for filename in glob.glob(os.path.join(Mal_path, '*.png')):
    img = cv2.imread(filename, 0)
    hist = hist = hog.compute(img)
    samples.append(hist)
    labels.append(1)

# Get negative samples
for filename in glob.glob(os.path.join(No_path, '*.png')):
    img = cv2.imread(filename, 0)
    hist = hist = hog.compute(img)
    samples.append(hist)
    labels.append(0)

# Convert objects to Numpy Objects
samples = np.float32(samples)
labels = np.array(labels)


# Shuffle Samples
rand = np.random.RandomState(321)
shuffle = rand.permutation(len(samples))
samples = samples[shuffle]
labels = labels[shuffle]   

np.save('data/smalldata/processed/samples',samples)
np.save('data/smalldata/processed/labels',labels)
print(samples)

[[[0.        ]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.25013185]
  [0.05519157]
  [0.04745237]
  ...
  [0.03627395]
  [0.00718676]
  [0.00083881]]

 [[0.00331275]
  [0.        ]
  [0.00387968]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 ...

 [[0.00061205]
  [0.        ]
  [0.        ]
  ...
  [0.        ]
  [0.        ]
  [0.00070738]]

 [[0.        ]
  [0.00229744]
  [0.1374592 ]
  ...
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.27983534]
  [0.0120645 ]
  [0.02628934]
  ...
  [0.00132558]
  [0.00081323]
  [0.26609066]]]
